# Pipeline For Computing Complete Payload Data

This pipeline is created for the ease of users willing to generate the complete data on their own. There are few things that should be kept in mind before executing this. 

>1. You should have enough space in your hard drive before executing this notebook. Approximately you should have atleast 400GB of space for storing and saving results of PCAP files.
>2. This notebook is compatible with python version 3.7.13. 
>3. Developed parser is based on Scapy module. Make sure it is installed. 
>4. Code processing might requrie high RAM space, therefore if you are on low resources try other method. 


In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import LabelEncoder
from Functions.Pipeline import *

/opt/conda/lib/python3.9/site-packages/scapy/layers/ipsec.py:471: CryptographyDeprecationWarning: Blowfish has been deprecated
  cipher=algorithms.Blowfish,
/opt/conda/lib/python3.9/site-packages/scapy/layers/ipsec.py:485: CryptographyDeprecationWarning: CAST5 has been deprecated
  cipher=algorithms.CAST5,


#### There are three inputs for the pipeline:

>1. In_directory (in_dir) = The directory where PCAP files are stored. For UNSW there are two folders wheras for CICIDS there are five individual files.
>2. Out_directory (out_dir) = The directory where you want the outcome of the tool to be stored.
>3. Dataset Name= `UNSW` or `CICIDS`.
>4. Processed CSV File = The directory for combined and processed CSV file. For processing the CSV files navigate to `CSV_data_preprocessing` folder

In [2]:
in_dir = "/home/jovyan/wire/DataSets/UNSW-NB15/UNSW-NB15 - pcap files"
out_dir = "/home/jovyan/UNSW_results"
Dataset_name = "UNSW"
processed_csv_file = "/home/jovyan/payload-byte-data/UNSW-NB15_processed.csv"

In [ ]:
df = pipeline(in_dir, out_dir, Dataset_name, processed_csv_file)

2022-10-27 15:40:24,015 - INFO -Checking directory for files ......
2022-10-27 15:40:24,015 - INFO -Files found. Initiating PCAP Parsing.......
2022-10-27 15:40:24,016 - INFO -Parsing 22-1-2015 Files .........
2022-10-27 15:40:24,016 - INFO -Reading input file:  /home/jovyan/wire/DataSets/UNSW-NB15/UNSW-NB15 - pcap files/pcaps 22-1-2015/1.pcap
2022-10-27 15:40:46,275 - INFO -Done with 100000 Packets
2022-10-27 15:41:08,858 - INFO -Done with 200000 Packets
2022-10-27 15:41:31,533 - INFO -Done with 300000 Packets
2022-10-27 15:41:54,195 - INFO -Done with 400000 Packets
2022-10-27 15:42:17,226 - INFO -Done with 500000 Packets
2022-10-27 15:42:39,544 - INFO -Done with 600000 Packets
2022-10-27 15:43:02,553 - INFO -Done with 700000 Packets
2022-10-27 15:43:24,965 - INFO -Done with 800000 Packets
2022-10-27 15:43:48,095 - INFO -Done with 900000 Packets
2022-10-27 15:44:11,671 - INFO -Done with 1000000 Packets
2022-10-27 15:44:34,701 - INFO -Done with 1100000 Packets
2022-10-27 15:44:58,401 -

tcp     1772972
udp       27125
ospf        254
arp         254
icmp         69
sctp          4
igmp          2
Name: protocol_m, dtype: int64


2022-10-27 15:48:16,593 - INFO -Done with 100000 Packets
2022-10-27 15:48:39,364 - INFO -Done with 200000 Packets
2022-10-27 15:49:02,368 - INFO -Done with 300000 Packets
2022-10-27 15:49:25,836 - INFO -Done with 400000 Packets
2022-10-27 15:49:49,351 - INFO -Done with 500000 Packets
2022-10-27 15:50:13,034 - INFO -Done with 600000 Packets
2022-10-27 15:50:35,672 - INFO -Done with 700000 Packets
2022-10-27 15:50:58,710 - INFO -Done with 800000 Packets
2022-10-27 15:51:22,218 - INFO -Done with 900000 Packets
2022-10-27 15:51:44,882 - INFO -Done with 1000000 Packets
2022-10-27 15:52:07,487 - INFO -Done with 1100000 Packets
2022-10-27 15:52:31,129 - INFO -Done with 1200000 Packets
2022-10-27 15:52:53,721 - INFO -Done with 1300000 Packets
2022-10-27 15:53:17,413 - INFO -Done with 1400000 Packets
2022-10-27 15:53:39,841 - INFO -Done with 1500000 Packets
2022-10-27 15:54:02,438 - INFO -Done with 1600000 Packets
2022-10-27 15:54:05,829 - INFO -Done reading /home/jovyan/wire/DataSets/UNSW-NB15

tcp       1589743
udp         24668
ospf          304
arp           188
icmp           54
pim             8
sctp            6
igmp            4
sep             2
swipe           1
mobile          1
sun-nd          1
Name: protocol_m, dtype: int64


2022-10-27 15:55:06,733 - INFO -Done with 100000 Packets
2022-10-27 15:55:29,819 - INFO -Done with 200000 Packets
2022-10-27 15:55:52,819 - INFO -Done with 300000 Packets
2022-10-27 15:56:15,904 - INFO -Done with 400000 Packets
2022-10-27 15:56:39,011 - INFO -Done with 500000 Packets
2022-10-27 15:57:02,206 - INFO -Done with 600000 Packets
2022-10-27 15:57:24,819 - INFO -Done with 700000 Packets
2022-10-27 15:57:48,205 - INFO -Done with 800000 Packets
2022-10-27 15:58:10,846 - INFO -Done with 900000 Packets
2022-10-27 15:58:34,513 - INFO -Done with 1000000 Packets
2022-10-27 15:58:57,178 - INFO -Done with 1100000 Packets
2022-10-27 15:59:19,891 - INFO -Done with 1200000 Packets
2022-10-27 15:59:43,604 - INFO -Done with 1300000 Packets
2022-10-27 16:00:06,284 - INFO -Done with 1400000 Packets
2022-10-27 16:00:28,983 - INFO -Done with 1500000 Packets
2022-10-27 16:00:53,114 - INFO -Done with 1600000 Packets


In [8]:
df.attack_cat.value_counts()

normal            6681484
generic             16829
exploits            13166
fuzzers             11898
reconnaissance       6812
dos                  2571
shellcode            1088
backdoor              519
analysis              383
worms                  93
Name: attack_cat, dtype: int64

## Undersampling Normal Data Instances

Since number of normal data instances are extensively higher than the attacks, normal instances are undersampled as mentioned in the paper. If you dont want to reduce the data instances ignore this step.

Or if you want to reduce it according to your approach change the data instances provided in `dict`.



In [ ]:
## For UNSW
dict = {
    "generic": 17580,
    "exploits": 13992,
    "fuzzers": 12722,
    "reconnaissance": 7562,
    "dos": 3397,
    "backdoor": 1239,
    "analysis": 1208,
    "shellcode": 1088,
    "normal": 21000,
    "worms": 93,
}

In [ ]:
## For CICIDS
dict = {
    "BENIGN": 362108,
    "DoS Hulk": 250000,
    "DDoS": 241405,
    "DoS GoldenEye": 128122,
    "DoS slowloris": 121097,
    "Infiltration": 115007,
    "DoS Slowhttptest": 80542,
    "SSH-Patator": 48165,
    "FTP-Patator": 31843,
    "Heartbleed": 13486,
    "Web Attack – Brute Force": 11754,
    "Web Attack – XSS": 3341,
    "Bot": 2543,
    "PortScan": 830,
    "Web Attack – Sql Injection": 12,
}

In [ ]:
X_res = df.iloc[:, :-1]
y_res = df.iloc[:, -1]

In [ ]:
rus = RandomUnderSampler(random_state=42, sampling_strategy=dict)
X_res, y_res = rus.fit_resample(df.iloc[:, :-1], df.iloc[:, -1])
X_res["label"] = y_res
df = 0
df = X_res

## Transformation of Hex Valued Payload into Byte-Wise Integers

Transform data into 1504 features, following the employed feature vector as explained in the paper.
Each feature is in integer form and can be utilized for training of Machine Learning models.

In [ ]:
# Out_dir => directory for saving the transformed data
out_dir = "D:/UNSW_results/"

df_t = transform(df, out_dir)